# モデルの最適化  

## トレーニング後量子化 - Post Training Quantization : PTQ  

ここでは、先ほどの学習済みResnet50モデルをINT8量子化することでモデルサイズを小さくし、推論パフォーマンスを向上させる方法について学びます。  

#### NNCF - Neural Network Compression Framework
モデルの量子化にはOpenVINOの関連ツール[NNCF](https://github.com/openvinotoolkit/nncf)を使用します。  
NNCFはQATとPTQの2つのモードでモデルの最適化を行うことができるツールです。**ここでは学習済みモデルの量子化を行うのでPTQモードを使用します**。    
NNCFはOpenVINOパッケージには含まれていません。別途インストールを行う必要があります。 
```sh
pip install nncf
```
使用するフレームワークも一緒にインストールするにはオプションを追加します。
```sh
pip install nncf[torch,tensorflow,onnx]
```

|モード|説明|
|---|---|
|[QAT : Quantize Aware Training](https://docs.openvino.ai/2024/openvino-workflow/model-optimization-guide/compressing-models-during-training.html)|モデルの学習中に量子化を行います。そのため、推論精度を犠牲にすることなくモデルの量子化を行うことが可能です。|
|[PTQ : Post Training Quantization](https://docs.openvino.ai/2024/openvino-workflow/model-optimization-guide/quantizing-models-post-training.html)|学習が終わったモデルデータを量子化します。ある程度の推論精度劣化を伴います。<br>アクティベーションレイヤーの統計データを元に量子化を行うために推論入力データが必要となります。通常は学習時に使用したデータセットのバリデーションデータなどを流用します。<br>また、[推論精度の劣化程度をコントロールすることができるモード](https://docs.openvino.ai/2024/openvino-workflow/model-optimization-guide/quantizing-models-post-training/quantizing-with-accuracy-control.html)もあります。|


## キャリブレーション用データセットのダウンロード  

'tiny-imagenet-200'をダウンロードし、展開します。(242MB)  
このデータセットに含まれる画像は64x64 (64,64,3)ですが、そのまま使用します。  

PTQを行うには入力データ(今回は画像分類モデル、Resnet50をPTQするので画像データ)が必要になります。

In [ ]:
import urllib.request
import shutil
import os

if not os.path.exists('tiny-imagenet-200.zip'):
    urllib.request.urlretrieve('http://cs231n.stanford.edu/tiny-imagenet-200.zip', 'tiny-imagenet-200.zip')
if not os.path.exists('tiny-imagenet-200'):
    shutil.unpack_archive('tiny-imagenet-200.zip')

## 必要なライブラリのインポート

In [ ]:
import glob
import cv2
import numpy as np

import openvino as ov

import torch
import torchvision

import nncf

## PyTorchのResnet50モデルを読み込む  

PyTorchの学習済みResnet50モデルを読み込みます。  
また、後のパフォーマンス比較で使うため、FP16のOpenVINO IRモデルに変換し、`resnet50_fp16.xml`という名前で保存します。

In [ ]:
resnet50 = torchvision.models.resnet50(weights=torchvision.models.ResNet50_Weights.DEFAULT)
resnet50.eval()

ov_model = ov.convert_model(resnet50, example_input=torch.rand(1,3,224,224).cpu())
ov.save_model(ov_model, 'resnet50_fp16.xml', compress_to_fp16=True)

## NNCFでPTQ(トレーニング後最適化)を実施するためのデータセットを準備する  

PyTorchなどで用意されているデータセットをそのまま利用できる場合は、それを利用したほうが簡単な場合もあります。  
[Post-Training Quantization of MobileNet v2 PyTorch Model](https://github.com/openvinotoolkit/nncf/tree/develop/examples/post_training_quantization/torch/mobilenet_v2)

ここでは、自分で用意したキャリブレーションデータを利用する場合を想定して、キャリブレーションデータセットを用意する簡易なコードを用意しました。

In [ ]:
resnet50 = torchvision.models.resnet50(weights=torchvision.models.ResNet50_Weights.DEFAULT)
resnet50.eval()

# モデルの入力に合わせて読み込んだ画像データをプリプロセスする
def preprocess(img, scale_val=255.0, mean_val=(0.485, 0.456, 0.406), std_val=(0.229, 0.224, 0.225)):
    img = cv2.resize(img, (224,224))                   # 224x224にリサイズ
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)         # BGR -> RGB
    img = (img.astype(np.float32) / 255.0) - 0.5       # 正規化 (0-255) -> (-0.5 - +0.5) 
    img = np.transpose(img, (2,0,1))                   # 軸の入れ替え HWC -> CHW
    return img                                         # この時点で (3,224,224)になっている

# 用意したデータローダーから与えられたデータアイテムからモデル入力データだけを抜き出すユーザー関数
# データアイテムは (img, label_id)が与えられることを想定。label_idは捨て、imgだけを抜き出す
def transform_fn(data_item):
    image, _ = data_item
    return image

# データセットオブジェクトを作成
# (image, label_id)のリスト
image_files = glob.glob('./tiny-imagenet-200/test/images/**/*.JPEG', recursive=True)     # データセット内にあるJPEGファイルパスのリストを取得
print(f'{len(image_files)} images found.')
dataset = [ (preprocess(cv2.imread(file_name)), 0) for file_name in image_files ]        # (image_data, label_id)で構成されるリストを作成する。label_idはダミー(0)固定

# データセットをデータローダーにセットし、キャリブレーションデータセットとする
val_loader = torch.utils.data.DataLoader(dataset, batch_size=1, shuffle=False, num_workers=0, pin_memory=True)
calibration_dataset = nncf.Dataset(val_loader, transform_fn)

## PTQ量子化を実行する  

量子化が終了後、量子化済みモデルをファイルに保存します。

In [ ]:
# PyTorchモデルの量子化実行
quantized_model = nncf.quantize(resnet50, calibration_dataset)

# 量子化済みモデルをOpenVINO IRモデルに変換し、保存
ov_model_int8 = ov.convert_model(quantized_model, example_input=torch.rand(1,3,224,224).cpu())
ov.save_model(ov_model_int8, 'resnet50_int8.xml')

## 量子化前のモデルと、量子化済みモデルのサイズを確認する  

`resnet50_fp16.bin`と`resnet50_int8.bin`のファイルサイズを確認してください。  
量子化したint8モデルのほうが小さくなっているはずです。

In [ ]:
if os.name == 'nt':
    !dir *.bin
else:
    !ls -l *.bin

## 量子化したモデルを実際に試してみる

## 量子化前(fp16)のモデルと量子化後(int8)のモデルをロードする

In [ ]:
device = 'CPU'
config = { 'CACHE_DIR' : './cache' }

compiled_model_fp16 = ov.compile_model('resnet50_fp16.xml', device, config=config)
compiled_model_int8 = ov.compile_model('resnet50_int8.xml', device, config=config)

## 入力画像の読み込みとプリプロセス

In [ ]:
from PIL import Image
from IPython.display import display

img = cv2.imread('beaver.jpg')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img = cv2.resize(img, (224,224))
display(Image.fromarray(img))
img = img.astype(np.float32) / 255.0 - 0.5
img = np.transpose(img, (2,0,1))
img = np.expand_dims(img, axis=0)
img.shape

## 量子化前(fp16)モデル  

モデルの推論結果と実行時間(wall time)を確認してください。

In [ ]:
%timeit compiled_model_fp16.infer_new_request(img)

with open('synset_words.txt') as f:
    imagenet_labels = [ line.rstrip().split(maxsplit=1)[1] for line in f.readlines() ]

result = compiled_model_fp16.infer_new_request(img)
res = result[0].ravel()
indices = np.argsort(res)[::-1]

for index in indices[:5]:
    print(index, imagenet_labels[index], res[index])

## 量子化前(int8)モデル  

モデルの推論結果と実行時間(wall time)を確認してください。fp16モデルとパフォーマンスを比較してください。

In [ ]:
%timeit compiled_model_int8.infer_new_request(img)

with open('synset_words.txt') as f:
    imagenet_labels = [ line.rstrip().split(maxsplit=1)[1] for line in f.readlines() ]

result = compiled_model_int8.infer_new_request(img)
res = result[0].ravel()
indices = np.argsort(res)[::-1]

for index in indices[:5]:
    print(index, imagenet_labels[index], res[index])

## まとめ  

ここではNNCFを使用してモデルの学習後量子化をする方法について学びました。  
NNCFは他にも学習時量子化や、プルーニングなど多くの機能があります。  
[OpenVINO Notebooks](https://openvinotoolkit.github.io/openvino_notebooks/)にも、モデル最適化に関する多くのチュートリアルがありますので参考にしてください。